In [5]:
pdf_directory = r'C:\Users\matthew\Documents\Science Fair 2017-2018\ManifestDownloads\PDFManifest'
import pyodbc
import re
import os

In [12]:
patients = []
for file in os.listdir(pdf_directory):
    if file[0:1] == 'T':
        content = ''
        f = open(pdf_directory + '\\' + file)
        for line in f:
            content += line
        content = content.lower()
        grade1 = 0
        grade2 = 0
        #print(re.findall(r'\d\s*[+]\s*\d', content))
        if len(re.findall(r'(\d)\s*[+]\s*(\d)', content)) == 1:
            grade1, grade2 = re.search(r'(\d)\s*[+]\s*(\d)', content).groups()
        elif len(re.findall(r'\d\s*[+]\s*\d', content)) > 1:
            if len(set(re.findall(r'\d\s*[+]\s*\d', content))) == 1:
                grade1, grade2 = re.search(r'(\d)\s*[+]\s*(\d)', content).groups()
            else:
                nums = []
                l = re.findall(r'\d\s*[+]\s*\d', content)
                s = list(set(l))
                for index in range(len(s)):
                    nums.append(0)
                    for grade in l:
                        if (grade == s[index]):
                            nums[index] += 1
                if sorted(nums)[-1] > sorted(nums)[-2]:
                    grade1, grade2 = re.search(r'(\d)\s*[+]\s*(\d)', s[nums.index(max(nums))]).groups()
        elif len(re.findall(r'primary\D{,20}\d', content)) == 1 and len(re.findall(r'secondary\D{,20}\d', content)) == 1:
            grade1 = re.search(r'primary\D{,20}(\d)', content).group(1)
            grade2 = re.search(r'secondary\D{,20}(\d)', content).group(1)
        patients.append((file, grade1, grade2))

In [17]:
server = r'localhost\SQLEXPRESS'
database = 'Patients'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = cnxn.cursor()
cnxn.autocommit = True

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 13 for SQL Server][SQL Server]Incorrect syntax near 'A'. (102) (SQLExecDirectW)")

In [19]:
for patient in patients:
    cursor.execute("update Patient set GleasonGradePrimary = {}, GleasonGradeSecondary = {} where EntityID = '{}'".format(
    patient[1], patient[2], patient[0]))